# covalent_api

> Endpoints for the `covalent` api: https://www.covalenthq.com/docs/api/

Summary:
    - We just *get* the data here - we process elsewhere.

In [ ]:
#|default_exp _covalent_api

In [ ]:
#| hide
from nbdev.showdoc import *

**Functionality of `Covalent_Api` class - this grabs data via the covalent API:**


`get_historical_balances`

Used to fetch the historical native, fungible (ERC20), and non-fungible (ERC721 & ERC1155) tokens held by an address at a given block height or date. Response includes daily prices and other metadata.
https://www.covalenthq.com/docs/api/balances/get-historical-token-balances-for-address/

`get_token_holders`

Get token holders as of any block height (v2).
Commonly used to get a list of all the token holders for a specified ERC20 or ERC721 token. Returns historic token holders when block-height is set (defaults to latest). Useful for building pie charts of token holders.
Note: This gets the holders by percentage (i.e. largest holdest to smallest)
https://www.covalenthq.com/docs/api/balances/get-token-holders-as-of-any-block-height-v2/


`get_holders_portfolios`

Wrapper around `get_historical_balances` to update a list of holders with their portfolios.




In [ ]:
#| export

import json
import requests
from requests.auth import HTTPBasicAuth
from requests.exceptions import HTTPError, RequestException
import time
from fastcore.basics import *
from fastcore.test import *
from typing import List

In [ ]:
#| export

import logging

# Set up basic logging configuration
logging.basicConfig(level=logging.INFO, format='%(message)s')

logger = logging.getLogger(__name__)

In [ ]:
#| export

import os
from dotenv import load_dotenv

load_dotenv()
COVALENT_API_KEY = os.environ.get('covalent_api_key')

Alright. Let's simplify things, and then generalise. And potentially refactor based on the endpoint changes. Whatever.

Here was how we did it before: modelled on the prior API docs.

In [ ]:
#Step 1) Let's expand the functionality of `get_historical_balances` to understand better what is going on. And look at the API docs etc:

#Authenticating with the API
headers = {
    "Authorization": f"Bearer {COVALENT_API_KEY}",
    "accept": "application/json"
}

#headers = {"accept": "application/json"}
#basic = HTTPBasicAuth(f'{COVALENT_API_KEY}', '')

#inputs to define url
chainName = 'eth-mainnet'
walletAddress = '0x7364a0f792e073814B426c918bf72792575b6c18'
quote_currency = 'USD'
date = '2023-10-20'

#Define the url
url = f"https://api.covalenthq.com/v1/{chainName}/address/{walletAddress}/historical_balances/?quote-currency={quote_currency}&date={date}"

#Make the request
#response = requests.get(url, headers=headers, auth=basic, timeout=10)
response = requests.get(url, headers=headers, timeout=10)
response.raise_for_status()
data = response.json() #this has keys: dict_keys(['data', 'error', 'error_message', 'error_code'])
data = data['data'] #dict_keys(['address', 'updated_at', 'next_update_at', 'quote_currency', 'chain_id', 'chain_name', 'items', 'pagination'])



In [ ]:
data

{'address': '0x7364a0f792e073814b426c918bf72792575b6c18',
 'updated_at': '2023-10-22T10:28:11.577264011Z',
 'next_update_at': '2023-10-22T10:33:11.577264566Z',
 'quote_currency': 'USD',
 'chain_id': 1,
 'chain_name': 'eth-mainnet',
 'items': [{'contract_decimals': 18,
   'contract_name': 'Revest',
   'contract_ticker_symbol': 'RVST',
   'contract_address': '0x120a3879da835a5af037bb2d1456bebd6b54d4ba',
   'supports_erc': ['erc20'],
   'logo_url': 'https://logos.covalenthq.com/tokens/1/0x120a3879da835a5af037bb2d1456bebd6b54d4ba.png',
   'block_height': 18387818,
   'last_transferred_block_height': 18097096,
   'last_transferred_at': '2023-09-09T06:36:47Z',
   'native_token': False,
   'type': 'cryptocurrency',
   'is_spam': False,
   'balance': '10642258892672793599819',
   'quote_rate': 0.0856,
   'quote': 910.9774,
   'pretty_quote': '$910.98',
   'nft_data': None},
  {'contract_decimals': 18,
   'contract_name': 'Ether',
   'contract_ticker_symbol': 'ETH',
   'contract_address': '0xee

Based on the current API docs, here is how we can do it now: 

In [ ]:
from covalent import CovalentClient

def main():
    c = CovalentClient("cqt_rQhtgfMTBwrVtKXGKHk3VWC6tMJ9")
    b = c.balance_service.get_historical_token_balances_for_wallet_address("eth-mainnet","0x7364a0f792e073814B426c918bf72792575b6c18", quote_currency="USD",date=2023-10-20)
    if not b.error:
        print(b.data)
    else:
        print(b.error_message)


In [ ]:
dir(CovalentClient)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [ ]:
c = CovalentClient("cqt_rQhtgfMTBwrVtKXGKHk3VWC6tMJ9")
b = c.balance_service.get_historical_token_balances_for_wallet_address("eth-mainnet","0x7364a0f792e073814B426c918bf72792575b6c18", quote_currency="USD",date='2023-10-20')



In [ ]:
CovalentClient??

Init signature: CovalentClient(api_key: str, debug: Optional[bool] = False)
Source:        
class CovalentClient:
    """ CovalentClient Class """

    security_service: SecurityService
  
    balance_service: BalanceService
  
    base_service: BaseService
  
    nft_service: NftService
  
    pricing_service: PricingService
  
    transaction_service: TransactionService
  
    xyk_service: XykService
  
    def __init__(self, api_key: str, debug: Optional[bool] = False):

        self.security_service = SecurityService(api_key, debug)
        self.balance_service = BalanceService(api_key, debug)
        self.base_service = BaseService(api_key, debug)
        self.nft_service = NftService(api_key, debug)
        self.pricing_service = PricingService(api_key, debug)
        self.transaction_service = TransactionService(api_key, debug)
        self.xyk_service = XykService(api_key, debug)
File:           ~/opt/anaconda3/envs/ether_crypto/lib/python3.10/site-packages/covalent/covalent_cl

In [ ]:
from covalent import CovalentClient


def main():
    c = CovalentClient(COVALENT_API_KEY, debug=True)
    b = c.balance_service.get_historical_token_balances_for_wallet_address("eth-mainnet","0x7364a0f792e073814B426c918bf72792575b6c18", quote_currency="USD",date='2023-10-20')
    if not b.error:
        print(b.data)
    else:
        print(b.error_message)
    return b


b=main()

[DEBUG] | Request URL: https://api.covalenthq.com/v1/eth-mainnet/address/0x7364a0f792e073814B426c918bf72792575b6c18/historical_balances/?quote-currency=USD&date=2023-10-20 | Response code: 200 | Response time: 556.49ms
None


In [ ]:
b.__dict__

{'data': None, 'error': True, 'error_code': None, 'error_message': None}

In [ ]:
import requests

url = "https://api.covalenthq.com/v1/eth-mainnet/address/0x7364a0f792e073814B426c918bf72792575b6c18/historical_balances/?quote-currency=USD&date=2023-10-20"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

print(response.json())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
b.error_message

In [ ]:
#| export

#The general API is: 
#                       def some_function(self,args):
#                            url = ... #define the url 
#                            return self.get_items(url)

#rather than writing self.get_items(url) every time, we can use a decorator to do this for us.
from requests.exceptions import HTTPError, ReadTimeout, RequestException

def url_decorator(func):
    def wrapper(self, *args, **kwargs):
        url = func(self, *args, **kwargs)
        return self.get_items(url)
    return wrapper

class Covalent_Api:
    """This class is used to interact with the Covalent API.
    """

    def __init__(self, covalent_api_key=COVALENT_API_KEY,
                 request_timeout=30,retries=5,delay=1
                 ):
        self.covalent_api_key = covalent_api_key
        self.request_timeout = request_timeout
        self.retries = retries
        self.delay = delay

    def get_items(self, url):
        """Given a url, get the items from the API."""

        headers = {"accept": "application/json"}
        basic = HTTPBasicAuth(f'{self.covalent_api_key}', '')

        while self.retries > 0:
            try:
                response = requests.get(url, headers=headers, auth=basic, timeout=self.request_timeout)
                response.raise_for_status()  
                data = response.json()['data']  # Directly access 'data' field
                
                if data is None:
                    logger.warning(f"No data received from {url}. Retries left: {self.retries}")
                    self.retries -= 1
                    time.sleep(self.delay)
                    continue

                if isinstance(data, list):
                    return data

                return data.get('items')

            except (ReadTimeout, RequestException, HTTPError) as e:
                logger.warning(f"{type(e).__name__} occurred: {e} for URL: {url}. Retrying in {self.delay} seconds...")
                self.retries -= 1
                time.sleep(self.delay)
                continue
            except Exception as e:
                logger.error(f"Unexpected error: {e} for URL: {url}. Returning None.")
                return None

        logger.warning(f"Exceeded the maximum number of retries ({self.retries}) for URL: {url} without success")
        return None

    @url_decorator
    def _fetch_historical_balances(self, chainName: str, walletAddress: str, date: str, quote_currency="USD") -> str:
        """
        This is an internal method that just constructs and returns the URL.
        """
        assert type(date) is str, "date must be a string"
        url = f"https://api.covalenthq.com/v1/{chainName}/address/{walletAddress}/historical_balances/?quote-currency={quote_currency}&date={date}"
        return url 

    def get_historical_balances(self, chainName: str, walletAddress: str, date: str, quote_currency="USD") -> list[dict]:
        """
        Given a wallet address, get the historical balances for that wallet address on the specified chain and date.
        This method fetches the data and then processes it.
        """
        items = self._fetch_historical_balances(chainName, walletAddress, date, quote_currency)
        if items is None:
            logger.error(f"Failed to fetch data for {walletAddress} on date {date}")
            return []
        
        return items 
    
    @url_decorator
    def get_token_holders(self, chainName: str, tokenAddress: str, block_height=None, page_size=100, page_number=None) -> list:
        """
        API docs: https://www.covalenthq.com/docs/api/balances/get-token-holders-as-of-any-block-height-v2/
        Fetches the token holders for a specific token on a given chain.
        Note: There is a possible `block-height` parameter, which we omit for now (see the API docs to clarify)
        
        Inputs:
            `chainName`: The chain name e.g. 'eth-mainnet'.
            `tokenAddress`: The token's address.
            `block_height`: Ending block to define a block range.
            `page_size`: Number of items per page. Supported values are 100 and 1000.
            `page_number`: 0-indexed page number to begin pagination.
        Outputs:
            `items`, a list of dictionaries, each dictionary containing information about a token holder.

        Note: This gets the holders by percentage (i.e. largest holdest to smallest)

        Note:
        While the core function generates a URL, the applied @url_decorator 
        modifies the return behavior to fetch items using that URL.
        """

        base_url = f"https://api.covalenthq.com/v1/{chainName}/tokens/{tokenAddress}/token_holders_v2/?"

        
        if block_height is not None:
            base_url += f"&block-height={block_height}"
        if page_size is not None:
            base_url += f"&page-size={page_size}"
        if page_number is not None:
            base_url += f"&page-number={page_number}"

        return base_url

    def get_holders_portfolios(self,wallet_list:List[dict],chainName:str,date:str,quote_currency="USD",log_output=False)->List[dict]:
        """Input: 
                `wallet_list`: a list of  wallets as dicts. Each dict must contain the key 'address'.
                 See get_historical_balances for description of other inputs.
           Output: `wallet_list` - the same list of dictionaries with the portfolio added.
        """

        #TODO: this address 0x80f8c8d0d29c99b7af1b4d97cad357061037ecb3 seems to give false data (~$20m for ShibaDoge
        #Maybe should email covalent about this to let them know. Please see the dextools chart for shibadoge. 
        #We plonked the info manually into the covalent API to verify and it looks like it is an issue on their end.
        #Just something to be aware of: covalent seems to not be infallible.

        # Setting up logging level based on the log_output value
        if log_output:
            logger.setLevel(logging.INFO)
        else:
            logger.setLevel(logging.WARNING)

        #loop over the list of dictionaries (holders of the token)
        for _holder in wallet_list: #this loop should be parallelized, but ok for now provided it isn't too big.

            _address = _holder['address']
            logger.info(f'getting {_address} portfolio')  # Logging statement in place of print
            _items = self.get_historical_balances(chainName=chainName,walletAddress=_address, quote_currency=quote_currency, date=date)
            if _items == None:
                print(f'failed to get {_address} portfolio: `get_historical_balances` returned `None`')  # Logging statement in place of print
                _holder['portfolio']=None
                _holder['portfolio_sum']=None
                continue

            #TODO: Make this more efficient: possibly convert to numpy array or dataframe or something and then sum
            
            none_to_zero = lambda x: 0 if x is None else x
            portfolio_sum = sum([none_to_zero(holding['quote']) for holding in _items])
            
            _holder['portfolio']=_items #update the _holder with the whole portfolio
            _holder['portfolio_sum']=portfolio_sum #update the _holder with the total value of the portfolio

            logger.info(f'got {_address} portfolio')  # Logging statement in place of print
            
        return wallet_list
    

In [ ]:
#Prototyping 
###Setup: hyperparameters
cov_api = Covalent_Api() #might need to pass an API key (different to default -- see __init__ of class)
#tokenAddress = '0x72e4f9F808C49A2a61dE9C5896298920Dc4EEEa9' #an erc20 token address (hpbitcoin)
chainName='eth-mainnet'
###


#New args required:
walletAddress = '0x7364a0f792e073814B426c918bf72792575b6c18'
date = '2023-08-30' #a date

#Get the historical balances for a given wallet address on a given chain and date
_holdings = cov_api.get_historical_balances(chainName=chainName,walletAddress=walletAddress,date=date) #This is a list of dicts

print(_holdings)

[{'contract_decimals': 18, 'contract_name': '2DAI.io', 'contract_ticker_symbol': '2DAI', 'contract_address': '0xb44b653f147569d88a684cbf6549e1968e8b2a1d', 'supports_erc': ['erc20'], 'logo_url': 'https://logos.covalenthq.com/tokens/1/0xb44b653f147569d88a684cbf6549e1968e8b2a1d.png', 'block_height': 18023703, 'last_transferred_block_height': 18005101, 'last_transferred_at': '2023-08-27T09:26:47Z', 'native_token': False, 'type': 'cryptocurrency', 'is_spam': False, 'balance': '610665846022468007830200', 'quote_rate': 0.0015497, 'quote': 946.3489, 'pretty_quote': '$946.35', 'nft_data': None}, {'contract_decimals': 18, 'contract_name': 'EverMoon', 'contract_ticker_symbol': 'EVERMOON', 'contract_address': '0x4ad434b8cdc3aa5ac97932d6bd18b5d313ab0f6f', 'supports_erc': ['erc20'], 'logo_url': 'https://logos.covalenthq.com/tokens/1/0x4ad434b8cdc3aa5ac97932d6bd18b5d313ab0f6f.png', 'block_height': 18023703, 'last_transferred_block_height': 17760310, 'last_transferred_at': '2023-07-24T03:33:23Z', 'nat

In [ ]:
_holdings[1]

{'contract_decimals': 18,
 'contract_name': 'EverMoon',
 'contract_ticker_symbol': 'EVERMOON',
 'contract_address': '0x4ad434b8cdc3aa5ac97932d6bd18b5d313ab0f6f',
 'supports_erc': ['erc20'],
 'logo_url': 'https://logos.covalenthq.com/tokens/1/0x4ad434b8cdc3aa5ac97932d6bd18b5d313ab0f6f.png',
 'block_height': 18023703,
 'last_transferred_block_height': 17760310,
 'last_transferred_at': '2023-07-24T03:33:23Z',
 'native_token': False,
 'type': 'cryptocurrency',
 'is_spam': False,
 'balance': '90517462125132589232055',
 'quote_rate': 0.0023993,
 'quote': 217.17854,
 'pretty_quote': '$217.18',
 'nft_data': None}

**PROBLEM**

Currently it does not successfully get all holdings. e.g. for my own metamask wallet,
it correctly retrieves that there is a holding (as of time of writing, Oct 2023) `Baconator`
however, it does not retrieve price data (i.e. `quote` is `None`). However, we can get 
this quantity through other APIs, e.g. `moralis` - which is slow and has cost, or `zerion`
which (atm) is free, but also slow per API call.

Anyway. 

How to use:

In [ ]:
###Setup: hyperparameters
cov_api = Covalent_Api() #might need to pass an API key (different to default -- see __init__ of class)
tokenAddress = '0x72e4f9F808C49A2a61dE9C5896298920Dc4EEEa9' #an erc20 token address (hpbitcoin)
chainName='eth-mainnet'
###

**How to use `get_token_holders`**: it is used to get the top token holders of a token.

In [ ]:
#get the top 100 holders of the given address
token_holders_hpbitcoin = cov_api.get_token_holders(chainName=chainName,tokenAddress=tokenAddress,page_size=100,page_number=0)
print(type(token_holders_hpbitcoin))
assert len(token_holders_hpbitcoin)==100
top_10_holders_hpbitcoin = token_holders_hpbitcoin[:10]
assert len(top_10_holders_hpbitcoin)==10

<class 'list'>


**How to use `get_historical_balances`**:

#NOTE: Basically a helper function to compute `get_holders_portfolios`

In [ ]:
#New args required:
walletAddress = top_10_holders_hpbitcoin[3]['address'] #a wallet address.
date = '2023-08-30' #a date

#Get the historical balances for a given wallet address on a given chain and date
portfolio = cov_api.get_historical_balances(chainName=chainName,walletAddress=walletAddress,date=date)

In [ ]:
#TODO: tests of `get_historical_balances` here

**How to use `get_holders_portfolios`**: requires a list of holders, e.g. output from `get_token_holders`. Basically it will update the token holders with their portfolios on the provided date (so a wrapper for `get_historical_balances`):

In [ ]:
#now update token holders info by getting their whole portfolios on the given `date`. We take the list output from `get_token_holders` and update it with the portfolio info.
date = '2023-08-30'
top_10_holders_hpbitcoin = cov_api.get_holders_portfolios(wallet_list=top_10_holders_hpbitcoin,chainName=chainName,date=date)

#Can now also get information like: what was the total portfolio value of the 4th largest holder of hpbitcoin on the given date?
print(f"Total portfolio value of the 4th largest holder of hpbitcoin on {date} was: ${top_10_holders_hpbitcoin[3]['portfolio_sum']} USD")
#top_10_holders_hpbitcoin[3]['portfolio'] gives the whole portfolio of the 4th largest holder of hpbitcoin on the given date.


Total portfolio value of the 4th largest holder of hpbitcoin on 2023-08-30 was: $2190284.3138066335 USD


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()